In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# New imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/Rovidicus/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# By defining main search values ahead of time, we'll streamline our code
# set our API call parameters 
LOCATION = 'Seattle, WA'
TERM = 'Italian'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "json files/results_seattle_italian.json"
JSON_FILE

'json files/results_seattle_italian.json'

In [6]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [7]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] json files/results_seattle_italian.json already exists. Deleting previous file...
[i] json files/results_seattle_italian.json not found. Saving empty list to new file.
- 0 previous results found.


65

In [8]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/65 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [9]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VOPdG8llLPaga9iJxXcMuQ,the-pink-door-seattle-4,The Pink Door,https://s3-media4.fl.yelpcdn.com/bphoto/sdrxZH...,False,https://www.yelp.com/biz/the-pink-door-seattle...,6932,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 47.61028, 'longitude': -122.3425}",[pickup],$$,"{'address1': '1919 Post Alley', 'address2': ''...",+12064433241,(206) 443-3241,1755.338520
1,XoLfRZ0jA8cf_MYL90OgEA,cascina-spinasse-seattle,Cascina Spinasse,https://s3-media4.fl.yelpcdn.com/bphoto/sP1YaA...,False,https://www.yelp.com/biz/cascina-spinasse-seat...,1353,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 47.615054, 'longitude': -122.314429}",[delivery],$$$,"{'address1': '1531 14th Ave', 'address2': '', ...",+12062517673,(206) 251-7673,1953.193674
2,lVSi-ilqM8FzT_HnK7cQ4A,il-terrazzo-carmine-seattle,Il Terrazzo Carmine,https://s3-media4.fl.yelpcdn.com/bphoto/6H3Nk0...,False,https://www.yelp.com/biz/il-terrazzo-carmine-s...,614,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 47.598694662445, 'longitude': -12...",[delivery],$$$,"{'address1': '411 1st Ave S', 'address2': '', ...",+12064677797,(206) 467-7797,2972.392938
3,dCwO2S65LZHuPg-WkKVajQ,brunello-ristorante-seattle,Brunello Ristorante,https://s3-media2.fl.yelpcdn.com/bphoto/TQR2td...,False,https://www.yelp.com/biz/brunello-ristorante-s...,377,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 47.68065, 'longitude': -122.31765}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '7115 Roosevelt Way NE', 'address...",+12065274778,(206) 527-4778,6280.369408
4,CB8GBAr5aUA_0Zii7fUsvQ,cotto-belltown-seattle-2,Cotto Belltown,https://s3-media3.fl.yelpcdn.com/bphoto/CCFRYV...,False,https://www.yelp.com/biz/cotto-belltown-seattl...,11,"[{'alias': 'italian', 'title': 'Italian'}]",5.0,"{'latitude': 47.61460332336834, 'longitude': -...",[],NaN,"{'address1': '2318 2nd Ave', 'address2': None,...",+12066447022,(206) 644-7022,1440.509019


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,gGMiFvQJ_c2FUvqaADBH8A,kitchen-and-market-seattle,Kitchen & Market,https://s3-media2.fl.yelpcdn.com/bphoto/8nJATq...,False,https://www.yelp.com/biz/kitchen-and-market-se...,38,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.5,"{'latitude': 47.61027, 'longitude': -122.34296}",[],NaN,"{'address1': '1926 Pike Pl', 'address2': None,...",+12064415747,(206) 441-5747,1773.185081
996,SJTPzzir65kTfCrA3-tVqw,little-caesars-pizza-burien,Little Caesars Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/uB0e5Y...,False,https://www.yelp.com/biz/little-caesars-pizza-...,27,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 47.47356, 'longitude': -122.34644}",[delivery],$,"{'address1': '14400 Ambaum Blvd SW', 'address2...",+12062442108,(206) 244-2108,16909.920441
997,0lPPYWYRyCUOl0MYGi5VFg,pagliacci-pizza-seattle-15,Pagliacci Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/qhNo0p...,False,https://www.yelp.com/biz/pagliacci-pizza-seatt...,82,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.6904233, 'longitude': -122.367...",[delivery],$$,"{'address1': '851 NW 85th St', 'address2': '',...",+12067261717,(206) 726-1717,7614.049650
998,q8-7VkagU0D9IQ292dpwHg,casa-guerrero-lynnwood,Casa Guerrero,https://s3-media1.fl.yelpcdn.com/bphoto/ivWHe6...,False,https://www.yelp.com/biz/casa-guerrero-lynnwoo...,558,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 47.8493456017319, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '402 164th St SW', 'address2': 'S...",+14257419608,(425) 741-9608,25932.286883
999,eX1kymKTUDgSx9ZQrYf4Hg,reds-wine-bar-kent,Reds Wine Bar,https://s3-media4.fl.yelpcdn.com/bphoto/Fuz54E...,False,https://www.yelp.com/biz/reds-wine-bar-kent?ad...,126,"[{'alias': 'wine_bars', 'title': 'Wine Bars'}]",4.0,"{'latitude': 47.38371116308968, 'longitude': -...",[delivery],$$,"{'address1': '321 Ramsay Way', 'address2': 'St...",+12532467341,(253) 246-7341,27914.224551


In [10]:
# save the final results to a compressed csv
final_df.to_csv('json files/results_seattle_italian.csv.gz', compression='gzip',index=False)